In [1]:
# YOUR CODE HERE
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
url = "http://books.toscrape.com/"

# TODO: Use `requests` to do an HTTP request to fetch data located at that URL
# TODO: Create a `BeautifulSoup` instance with that data

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
type(soup)

bs4.BeautifulSoup

In [9]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

20

## Parsing _one_ book

In [10]:
books_html[0]

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

## Parsing one book name

In [12]:
book_title = books_html[0].find("h3").find("a")["title"]
book_title

'A Light in the Attic'

## Parsing one book price

In [16]:
book_price = books_html[0].find("p", class_="price_color").text
book_price

'£51.77'

In [ ]:
book_price = float(book_price[1:])
type(book_price)


float

## Parsing one book rating

In [28]:
book_stars_html = books_html[0].find("p", class_="star-rating")
book_stars_html

<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>

In [55]:
rating_class = book_stars_html.attrs['class'][1]
rating_class

'Three'

In [ ]:
def parse_rating(book_stars_html):
        if rating_class == "One":
            return 1
        if rating_class == "Two":
            return 2
        if rating_class == "Three":
            return 3
        if rating_class == "Four":
            return 4
        else:
            return 5


In [57]:
book_rating = parse_rating(book_stars_html)
book_rating

3

## Parsing _all_ books

In [58]:
books_dict = { 'Title': [], 'Price': [], 'Rating': [] }

In [ ]:
for books_html in soup.find_all("article", class_="product_pod"):
    book_title = books_html.find("h3").find("a")["title"],
    book_price = books_html.find("p", class_="price_color").text.strip("£"),
    book_stars_html = books_html.find("p", class_="star-rating"),
    book_rating = parse_rating(book_stars_html),
    books_dict["Title"].append(book_title),
    books_dict["Price"].append(book_price),
    books_dict["Rating"].append(book_rating)
books_dict


{'Title': [('A Light in the Attic',),
  ('Tipping the Velvet',),
  ('Soumission',),
  ('Sharp Objects',),
  ('Sapiens: A Brief History of Humankind',),
  ('The Requiem Red',),
  ('The Dirty Little Secrets of Getting Your Dream Job',),
  ('The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',),
  ('The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',),
  ('The Black Maria',),
  ('Starving Hearts (Triangular Trade Trilogy, #1)',),
  ("Shakespeare's Sonnets",),
  ('Set Me Free',),
  ("Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",),
  ('Rip it Up and Start Again',),
  ('Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',),
  ('Olio',),
  ('Mesaerion: The Best Science Fiction Stories 1800-1849',),
  ('Libertarianism for Beginners',),
  ("It's Only the Himalayas",)],
 'Price': [('51.77',),
  ('53.74',),
  ('50.10',),
  ('47.82',),
  ('54.23',),
  ('22.65',),
  ('33.34'

In [64]:
len(books_dict)          # 3 key:value çifti olmalı

3

In [65]:
len(books_dict["Title"]) # Her listede 20 kitap olmalı

20

In [66]:
books_dict = pd.DataFrame(books_dict)
books_dict.head()

,Title,Price,Rating
0,"(A Light in the Attic,)","(51.77,)","(3,)"
1,"(Tipping the Velvet,)","(53.74,)","(3,)"
2,"(Soumission,)","(50.10,)","(3,)"
3,"(Sharp Objects,)","(47.82,)","(3,)"
4,"(Sapiens: A Brief History of Humankind,)","(54.23,)","(3,)"


In [67]:
books_dict = books_dict.applymap(
    lambda x: x[0] if isinstance(x, tuple) else x
)
books_dict.head()

/var/folders/cz/4fnqmsqn071df7ryv9gz5tzr0000gn/T/ipykernel_19392/3896498510.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  books_dict = books_dict.applymap(


,Title,Price,Rating
0,A Light in the Attic,51.77,3
1,Tipping the Velvet,53.74,3
2,Soumission,50.10,3
3,Sharp Objects,47.82,3
4,Sapiens: A Brief History of Humankind,54.23,3


In [76]:
books_dict["Price"] = books_dict["Price"].astype(float)


In [78]:
from nbresult import ChallengeResult

result = ChallengeResult('books',
    columns=books_dict.columns,
    title=str(books_dict.loc[0,'Title']),
    price=books_dict.loc[0,'Price'],
    rating=books_dict.loc[0,'Rating']
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/elifcubukcu/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/elifcubukcu/data-book-scraping/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 5 items

test_books.py::TestBooks::test_books_df_columns_name PASSED              [ 20%]
test_books.py::TestBooks::test_books_df_columns_size PASSED              [ 40%]
test_books.py::TestBooks::test_first_book_price PASSED                   [ 60%]
test_books.py::TestBooks::test_first_book_rating PASSED                  [ 80%]
test_books.py::TestBooks::test_first_book_title PASSED                   [100%]

============================== 5 passed in 0.58s ===============================


💯 You can commit your code:

git add tests/books.pickle

git commit -m 'Completed books step'

git push origin master

